In [1]:
import findspark 
findspark.init('/home/abraham/spark-2.2.1-bin-hadoop2.7') 
import pyspark
import os
import sys
from pyspark.sql import SQLContext
from pyspark import SparkContext
sc =SparkContext()
sqlContext = SQLContext(sc) 

In [2]:
#flights = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/home/abraham/MGE_2018/tarea4/flights.csv')

In [2]:
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('cruise').getOrCreate() 
flights = spark.read.csv('/home/abraham/MGE_2018/tarea4/flights.csv',inferSchema=True,header=True)

In [3]:
flights=flights.na.fill(0)
flights=flights.na.fill("?")

In [4]:
flights.first

<bound method DataFrame.first of DataFrame[YEAR: int, MONTH: int, DAY: int, DAY_OF_WEEK: int, AIRLINE: string, FLIGHT_NUMBER: int, TAIL_NUMBER: string, ORIGIN_AIRPORT: string, DESTINATION_AIRPORT: string, SCHEDULED_DEPARTURE: int, DEPARTURE_TIME: int, DEPARTURE_DELAY: int, TAXI_OUT: int, WHEELS_OFF: int, SCHEDULED_TIME: int, ELAPSED_TIME: int, AIR_TIME: int, DISTANCE: int, WHEELS_ON: int, TAXI_IN: int, SCHEDULED_ARRIVAL: int, ARRIVAL_TIME: int, ARRIVAL_DELAY: int, DIVERTED: int, CANCELLED: int, CANCELLATION_REASON: string, AIR_SYSTEM_DELAY: int, SECURITY_DELAY: int, AIRLINE_DELAY: int, LATE_AIRCRAFT_DELAY: int, WEATHER_DELAY: int]>

In [5]:
columnList = [item[0] for item in flights.dtypes if item[1].startswith('string')]
columnList
#flights.columns

['AIRLINE',
 'TAIL_NUMBER',
 'ORIGIN_AIRPORT',
 'DESTINATION_AIRPORT',
 'CANCELLATION_REASON']

# Data Partition

In [6]:
# separamos los datos en entrenamiento y pruebas (70 y 30)
#(training_data, test_data) = flights.randomSplit([0.7, 0.3])

In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import MinMaxScaler

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(flights) for column in 
            list(set(columnList)) ]


In [9]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(flights).transform(flights) 
df_r=assembler.transform(df_r)
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features") 
scaler_model = scaler.fit(df_r).transform(df_r) 


NameError: name 'assembler' is not defined

In [13]:
def sels(base):
    sels=base.select("scaled_features", "DEPARTURE_DELAY") 
    return(sels)

In [14]:
pipe=Pipeline(stages=[pipeline,assembler,scaler])
dff=pipe.fit(flights).transform(flights)

NameError: name 'assembler' is not defined

In [ ]:

sels(dff).show(1)

In [15]:
assembler = VectorAssembler(
  inputCols=[#'YEAR',
             #'MONTH',
             #'DAY',
             'DAY_OF_WEEK',
             #'AIRLINE',
             #'FLIGHT_NUMBER',
            # 'TAIL_NUMBER',
            #'ORIGIN_AIRPORT',
            #'DESTINATION_AIRPORT',
            #'SCHEDULED_DEPARTURE',
            #'DEPARTURE_TIME',
            'TAXI_OUT',
            'WHEELS_OFF',
            #'SCHEDULED_TIME',
            'ELAPSED_TIME',
            'AIR_TIME',
            'DISTANCE',
            'WHEELS_ON',
            'TAXI_IN',
            #'SCHEDULED_ARRIVAL',
            #'ARRIVAL_TIME',
            #'ARRIVAL_DELAY',
            #'DIVERTED',
            #'CANCELLED',
            #'CANCELLATION_REASON',
            #'AIR_SYSTEM_DELAY',
            #'SECURITY_DELAY',
            #'AIRLINE_DELAY','LATE_AIRCRAFT_DELAY','WEATHER_DELAY'
  ],
    outputCol="features")

In [16]:
output = assembler.transform(flights)

In [18]:
.show()

+--------------------+---------------+
|            features|DEPARTURE_DELAY|
+--------------------+---------------+
|[4.0,21.0,15.0,19...|            -11|
|[4.0,12.0,14.0,27...|             -8|
|[4.0,16.0,34.0,29...|             -2|
|[4.0,15.0,30.0,28...|             -5|
|[4.0,11.0,35.0,21...|             -1|
|[4.0,18.0,38.0,23...|             -5|
|[4.0,11.0,30.0,17...|             -6|
|[4.0,13.0,57.0,24...|             14|
|[4.0,17.0,36.0,19...|            -11|
|[4.0,12.0,45.0,20...|              3|
|[4.0,12.0,36.0,14...|             -6|
|[4.0,21.0,48.0,26...|             -8|
|[4.0,18.0,53.0,21...|              0|
|[4.0,18.0,52.0,19...|             -6|
|[4.0,28.0,107.0,1...|             -1|
|[4.0,17.0,58.0,19...|             -4|
|[4.0,25.0,56.0,20...|            -14|
|[4.0,11.0,53.0,21...|             -6|
|[4.0,11.0,57.0,20...|             -4|
|[4.0,9.0,54.0,186...|             -5|
+--------------------+---------------+
only showing top 20 rows



In [17]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in 
            list(set(df.columns)-set(['date'])) ]


NameError: name 'df' is not defined

In [18]:
pipeline = Pipeline(stages=[indexers,assembler])
df_r = pipeline.fit(flights).transform(flights) 

TypeError: Cannot recognize a pipeline stage of type <class 'list'>.

In [19]:
final_data=output.select("features", "DEPARTURE_DELAY")

In [20]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [26]:
from pyspark.ml.regression import LinearRegression,DecisionTreeRegressor,GeneralizedLinearRegression
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='DEPARTURE_DELAY')
glm=GeneralizedLinearRegression(labelCol='DEPARTURE_DELAY')
#rf.extractParamMap()

In [27]:
glmmodel=glm.fit(train_data)

In [25]:
dt.extractParamMap()

{Param(parent='DecisionTreeRegressor_4c3590085ed0edd7ab29', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 5,
 Param(parent='DecisionTreeRegressor_4c3590085ed0edd7ab29', name='maxBins', doc='Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature.'): 32,
 Param(parent='DecisionTreeRegressor_4c3590085ed0edd7ab29', name='minInstancesPerNode', doc='Minimum number of instances each child must have after split. If a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.'): 1,
 Param(parent='DecisionTreeRegressor_4c3590085ed0edd7ab29', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users 

In [22]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [25]:
#NEW=flights.select('DISTANCE',"DEPARTURE_DELAY").na.fill(0)
test_results = lrModel.evaluate(test_data)

In [26]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
#from pyspark.sql.functions import when, mean 
#new2=NEW.withColumn('DEPARTURE_DELAY',when(NEW.DEPARTURE_DELAY.isNull(),NEW.DEPARTURE_DELAY.mean()).otherwise(NEW.DEPARTURE_DELAY))

RMSE: 36.19759466557666


In [24]:
#NEW.show(100)

In [29]:
from pyspark.ml.regression import RandomForestRegressor

In [32]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features",labelCol="DEPARTURE_DELAY")

In [33]:
rfmodel = rf.fit(train_data)

In [37]:
test_rf = rfmodel.transform(test_data)

In [99]:
# parameter grid
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
param_grid = ParamGridBuilder().\
    addGrid(lr.regParam, [0, 0.5, 1]).\
    addGrid(lr.elasticNetParam, [0, 0.5, 1]).\
    build()

In [100]:
from pyspark.ml.evaluation import RegressionEvaluator 
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=param_grid,
                          evaluator=RegressionEvaluator(predictionCol='prediction', labelCol="DEPARTURE_DELAY", 
                               metricName='rmse'),#default es rmse
                          numFolds=2)
#Param for metric name in evaluation. Supports: - "rmse" (default): 
#root mean squared error - "mse": mean squared error - "r2": R^2^ metric - "mae": mean absolute error

In [101]:
cvModel = crossval.fit(train_data)

In [105]:
#https://mingchen0919.github.io/learning-apache-spark/linear-regression.html
cvModel.bestModel

LinearRegression_42538153e1b94eb04c82

In [109]:
min(cvModel.avgMetrics)

36.116920105333456

In [108]:
param_grid[2]

{Param(parent='LinearRegression_42538153e1b94eb04c82', name='regParam', doc='regularization parameter (>= 0).'): 0,
 Param(parent='LinearRegression_42538153e1b94eb04c82', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 1}

In [62]:
bestPipeline = cvModel.bestModel
#bestLRModel = bestPipeline.stages[2]
bestParams = bestPipeline.extractParamMap()

In [67]:
bestPipeline.extractParamMap()

{}

In [70]:
print ('Best Param (regParam): ', bestPipeline._java_obj.getRegParam()) 
print ('Best Param (elasticNetParam): ', bestPipeline._java_obj.getRegParam())

Best Param (regParam):  0.0
Best Param (elasticNetParam):  0.0


In [74]:
print ('Best Param (regParam): ', bestPipeline._java_obj.getRegParam()) 

Best Param (regParam):  0.0


In [77]:
rf.extractParamMap()

{Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations.'): 10,
 Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='minInfoGain', doc='Minimum information gain for a split to be considered at a tree node.'): 0.0,
 Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='featuresCol', doc='features column name.'): 'features',
 Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='impurity', doc='Criterion used for information gain calculation (case-insensitive). Supported options: variance'): 'variance',
 Param(parent='RandomForestRegressor_44219e4cf8e48e

In [168]:
# parameter grid
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
param_gridx = ParamGridBuilder().\
    addGrid(rf.numTrees, [10, 15, 20]).\
    build()

In [169]:
crossval = CrossValidator(estimator=rf,
                          estimatorParamMaps=param_gridx,
                          evaluator=RegressionEvaluator(predictionCol='prediction', labelCol="DEPARTURE_DELAY", 
                               metricName='rmse'),#default es rmse
                          numFolds=2)

In [170]:
cvModel = crossval.fit(train_data)

In [171]:
cvModel.bestModel

RandomForestRegressionModel (uid=RandomForestRegressor_44219e4cf8e48eded49a) with 20 trees

In [172]:
min(cvModel.avgMetrics)

35.585590710309674

In [95]:
param_gridx[2]

{Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='numTrees', doc='Number of trees to train (>= 1).'): 20}

In [113]:
rdd = sc.parallelize(param_gridx)

In [173]:
cvModel.avgMetrics.index(min(cvModel.avgMetrics))

2

In [174]:
%%time 
param_gridx[cvModel.avgMetrics.index(min(cvModel.avgMetrics))]

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.06 µs


{Param(parent='RandomForestRegressor_44219e4cf8e48eded49a', name='numTrees', doc='Number of trees to train (>= 1).'): 20}

In [164]:
def define_hyper_params():

    ## Por ejemplo
    # Create space of candidate learning algorithms and their hyperparameters
    modelo = {'lr': LinearRegression(labelCol='DEPARTURE_DELAY'),
    'rf': RandomForestRegressor(featuresCol="features",labelCol="DEPARTURE_DELAY")}
    
    search_space = [ParamGridBuilder().\
    addGrid(modelo['lr'].regParam, [0, 0.5, 1]).\
    addGrid(modelo['lr'].elasticNetParam, [0, 0.5, 1]).\
    build()
    ,
    ParamGridBuilder().\
    addGrid(modelo['rf'].numTrees, [10, 15, 20]).\
    build()]

    return (modelo,search_space)

In [144]:
modelo = {'lr': LinearRegression(labelCol='DEPARTURE_DELAY'),
'rf': RandomForestRegressor(featuresCol="features",labelCol="DEPARTURE_DELAY")}
modelo['rf']

RandomForestRegressor_428f8a927c9be1abcb60

In [163]:
classifier,search_space=define_hyper_params()
search_space[0][0]

{Param(parent='LinearRegression_4eea8199a2f65bdf0bfc', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0,
 Param(parent='LinearRegression_4eea8199a2f65bdf0bfc', name='regParam', doc='regularization parameter (>= 0).'): 0}

In [175]:
from pyspark.ml import Pipeline
def magic_loop(X_train,models_to_run=['lr','rf']):
    modelo,search_space=define_hyper_params() 
    best=[] 
    metr=[] 
    params=[] 
    for i in range(len(models_to_run)):
        pipeline = Pipeline(stages=[modelo[models_to_run[i]]]) 
        crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=search_space[i],
                          evaluator=RegressionEvaluator(predictionCol='prediction', labelCol="DEPARTURE_DELAY", 
                               metricName='rmse'),#default es rmse
                          numFolds=2)
        cvModel = crossval.fit(X_train)
        best_model=cvModel.bestModel
        best.append(best_model)
        metr.append(min(cvModel.avgMetrics))
        params.append(search_space[i][cvModel.avgMetrics.index(min(cvModel.avgMetrics))])
        print('Mejor modelo de ',models_to_run[i],'fue:',search_space[i][cvModel.avgMetrics.index(min(cvModel.avgMetrics))]) 
    
    print(metr)
    print('Mejor modelo fue:',params[metr.index(min(metr))]) 
            
    return(best[metr.index(min(metr))])

In [176]:
magic_loop(train_data)

Mejor modelo de  lr fue: {Param(parent='LinearRegression_484eb7b4928ee76a20b2', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0, Param(parent='LinearRegression_484eb7b4928ee76a20b2', name='regParam', doc='regularization parameter (>= 0).'): 0}
Mejor modelo de  rf fue: {Param(parent='RandomForestRegressor_401fbe5732ff2cfe2013', name='numTrees', doc='Number of trees to train (>= 1).'): 20}
[36.116920105333456, 35.58559071030968]
Mejor modelo fue: {Param(parent='RandomForestRegressor_401fbe5732ff2cfe2013', name='numTrees', doc='Number of trees to train (>= 1).'): 20}


PipelineModel_469dae43e856896c125e